In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import msmrd2
import msmrd2.tools.quaternions as quats
import msmrd2.visualization as msmrdvis
from msmrd2.potentials import patchyParticleAngular
from msmrd2.integrators import overdampedLangevin as odLangevin
import msmrd2.tools.quaternions as quats
import timeit

In [ ]:
# Main parameters
numparticles = 2
boxsize = 2 #5 
D = 1.0
Drot = 1.0
relativeDistanceCutOff = 2.2
numTrajectories = 1

In [ ]:
# Particle list generation
def randomPartList():
    '''
    Generates particle list with uniformly random position and orientation. It also enforces the particles don't
    overlap over a distance given by the relative distance cut off.
    '''
    pyPartlist = [] 
    for i in range(numparticles):

        overlap = True
        while overlap:
            position = np.array([boxsize*np.random.rand()-0.5*boxsize,
                                 boxsize*np.random.rand()-0.5*boxsize,
                                 boxsize*np.random.rand()-0.5*boxsize])
            overlap = False     
            for j in range(len(pyPartlist)):
                if np.linalg.norm(position - pyPartlist[j].position) < relativeDistanceCutOff:
                    overlap = True
                    continue

        orientation = np.array([np.random.rand(),np.random.rand(),np.random.rand(),np.random.rand()])
        orientation = orientation/np.linalg.norm(orientation)
        part = msmrd2.particle(D, Drot, position, orientation)
        pyPartlist.append(part)

    partlist = msmrd2.integrators.particleList(pyPartlist)
    return partlist

In [ ]:
# Define integrator and boundary (over-damped Langevin)
dt = 0.0001 #0.00001 #0.000005
seed = -1 # Negative seed, uses random device as seed
bodytype = 'rigidbody'
integrator = odLangevin(dt, seed, bodytype) 
boxBoundary = msmrd2.box(boxsize,boxsize,boxsize,'periodic')
integrator.setBoundary(boxBoundary)

In [ ]:
# Define Patchy Particle potential
sigma = 1.0
strength = 160 #200.0
angularStrength = 20 #200.0
angleDiff = 3*np.pi/5.0
patch1 = np.array([np.cos(angleDiff/2),np.sin(angleDiff/2),0.])
patch2 = np.array([np.cos(-angleDiff/2),np.sin(-angleDiff/2),0.])
patchesCoordinates = [patch1, patch2]
potentialPatchyParticleAngular = patchyParticleAngular(sigma, strength, angularStrength, patchesCoordinates)
integrator.setPairPotential(potentialPatchyParticleAngular)

In [ ]:
# Generates numTrajectories trajectories starting at a random position and ending in a given bound state.
# This yields the first passage times for a given bound state. Each trajectory is Integrated until 
# a bound state is reached. The output in the files is the elapsed time and final state.
traj = msmrd2.trajectories.patchyDimer(2,1) # dummy trajectory to calculate boundstate of patchydimer
for i in range(numTrajectories):
    partlist = randomPartList()
    unbound = True
    while(unbound):
        integrator.integrate(partlist)
        boundState = traj.getBoundState(partlist[0], partlist[1])
        if ( (boundState >= 1) and (boundState <= 8)):
            print(integrator.clock, boundState)
            print(partlist[0].position, partlist[0].orientation)
            print(partlist[1].position, partlist[1].orientation)
            integrator.resetClock()
            break;

In [ ]:
datafile  = open('../../data/vmd/tests/dimerStates.xyz', 'w')
datafile.write(str(3*len(partlist)) + '\n')
datafile.write(str(0) + '\n')
for j, part in enumerate(partlist):
    v0 = part.position
    v1 = v0 + 0.6*sigma*quats.rotateVec(patchesCoordinates[0], part.orientation)
    v2 = v0 + 0.6*sigma*quats.rotateVec(patchesCoordinates[1], part.orientation)
    if j == 0:
        stringtype = 'type_0'
    else:
        stringtype = 'type_1'
    datafile.write(stringtype + ' ' + ' '.join(map(str, v0)) + '\n')
    datafile.write('type_2' + ' ' + ' '.join(map(str, v1)) + '\n')
    datafile.write('type_3' + ' ' + ' '.join(map(str, v2)) + '\n')
msmrdvis.generateTCL_patchyProteinsV2(numparticles = numparticles, 
                                     outfname = "dimerStates", 
                                     tclfname = "../../data/vmd/tests/dimerStates2vmd.tcl")

In [ ]:
part1 = partlist[0]
part2 = partlist[1]
q1 = part1.orientation
q2 = part2.orientation
relquat = quats.multiply(quats.conjugate(q1), q2)
axisAng = quats.quat2angle(relquat)
print(relquat, axisAng)

In [ ]:
axisAng1 = quats.quat2angle(q1)
axisAng2 = quats.quat2angle(q2)
print(axisAng1, axisAng2)

In [ ]:
theta  = 2*np.pi*np.random.rand()
rotaxis = (np.random.rand(), np.random.rand(), np.random.rand())
rotaxis = rotaxis/np.linalg.norm(rotaxis)
rotaxis = theta*rotaxis
print(rotaxis)
quat = msmrd2.tools.angle2quat(rotaxis)
#print(quat)
backrot = msmrd2.tools.quat2angle(quat)
print(backrot)

In [ ]:
quat

In [ ]:
backrot = msmrd2.tools.quat2angle(quat)
backrot

In [ ]:
rotaxis*theta